# Data Preparation and Processing

In [1]:

import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler


In [2]:

train_samples = []
train_labels = []

In [3]:

for i in range(50):
    # 5% of younger individuals with side effects
    random_younger = randint(13, 64)
    train_samples.append(random_younger)
    train_labels.append(1)
    
    # 5% of older individuals with no side effects
    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(950):
    # 95% of younger individuals with no side effects
    random_younger = randint(13, 64)
    train_samples.append(random_younger)
    train_labels.append(0)
    
    # 5% of older individuals with no side effects
    random_older = randint(65, 100)
    train_samples.append(random_older)
    train_labels.append(1)
    

In [4]:
train_samples = np.array(train_samples)
train_labels = np.array(train_labels)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1, 1))

for i in scaled_train_samples:
    print(i)

# print(train_samples, train_labels)

[0.17241379]
[0.75862069]
[0.25287356]
[0.86206897]
[0.32183908]
[0.96551724]
[0.49425287]
[0.89655172]
[0.]
[0.62068966]
[0.08045977]
[0.62068966]
[0.08045977]
[0.94252874]
[0.2183908]
[0.95402299]
[0.5862069]
[0.70114943]
[0.26436782]
[1.]
[0.3908046]
[0.82758621]
[0.17241379]
[0.91954023]
[0.34482759]
[0.79310345]
[0.5862069]
[0.65517241]
[0.51724138]
[0.6091954]
[0.31034483]
[0.77011494]
[0.12643678]
[0.66666667]
[0.55172414]
[0.71264368]
[0.51724138]
[0.77011494]
[0.40229885]
[0.97701149]
[0.17241379]
[0.70114943]
[0.40229885]
[0.63218391]
[0.56321839]
[0.6091954]
[0.5862069]
[0.59770115]
[0.01149425]
[0.68965517]
[0.11494253]
[0.63218391]
[0.3908046]
[0.90804598]
[0.18390805]
[0.81609195]
[0.06896552]
[0.65517241]
[0.36781609]
[0.98850575]
[0.18390805]
[0.96551724]
[0.22988506]
[0.94252874]
[0.32183908]
[1.]
[0.42528736]
[0.91954023]
[0.24137931]
[0.85057471]
[0.]
[0.82758621]
[0.50574713]
[0.90804598]
[0.56321839]
[0.7816092]
[0.31034483]
[0.65517241]
[0.4137931]
[0.73563218]
[0

# Sequential Model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

2023-10-10 08:27:24.782323: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-10 08:27:25.825052: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-10 08:27:25.826223: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-10 08:27:25.830219: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-10 08:27:26.293020: I tensorflow/core/platform/cpu_feature_g

In [6]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print(f"num GPUs available: {len(physical_devices)}")

num GPUs available: 1


2023-10-10 08:27:32.367369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:32.457779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:32.457827: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [7]:
model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation="relu"),
    Dense(units=2, activation="softmax")
])

2023-10-10 08:27:32.601263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:32.601347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:32.601362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:39.307742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-10 08:27:39.308021: I tensorflow/compile

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                32        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 642 (2.51 KB)
Trainable params: 642 (2.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
model.fit(x=scaled_train_samples, y=train_labels, validation_split=0.1, batch_size=10, epochs=30, shuffle=True, verbose=2)

Epoch 1/30


2023-10-10 08:28:52.376336: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-10 08:28:53.000465: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b5b0560ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-10 08:28:53.000524: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-10-10 08:28:53.275061: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-10 08:28:56.611377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-10-10 08:28:56.771686: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


180/180 - 80s - loss: 0.6492 - accuracy: 0.5539 - val_loss: 0.6316 - val_accuracy: 0.6150 - 80s/epoch - 445ms/step
Epoch 2/30
180/180 - 1s - loss: 0.6284 - accuracy: 0.6061 - val_loss: 0.6068 - val_accuracy: 0.6550 - 744ms/epoch - 4ms/step
Epoch 3/30
180/180 - 1s - loss: 0.6061 - accuracy: 0.6661 - val_loss: 0.5806 - val_accuracy: 0.7200 - 683ms/epoch - 4ms/step
Epoch 4/30
180/180 - 1s - loss: 0.5835 - accuracy: 0.6961 - val_loss: 0.5542 - val_accuracy: 0.7400 - 653ms/epoch - 4ms/step
Epoch 5/30
180/180 - 1s - loss: 0.5600 - accuracy: 0.7411 - val_loss: 0.5269 - val_accuracy: 0.7800 - 679ms/epoch - 4ms/step
Epoch 6/30
180/180 - 1s - loss: 0.5357 - accuracy: 0.7728 - val_loss: 0.4984 - val_accuracy: 0.8150 - 691ms/epoch - 4ms/step
Epoch 7/30
180/180 - 1s - loss: 0.5109 - accuracy: 0.7922 - val_loss: 0.4698 - val_accuracy: 0.8400 - 653ms/epoch - 4ms/step
Epoch 8/30
180/180 - 1s - loss: 0.4864 - accuracy: 0.8200 - val_loss: 0.4412 - val_accuracy: 0.8750 - 585ms/epoch - 3ms/step
Epoch 9/30